In [1]:
!pip install strsimpy

     |████████████████████████████████| 45 kB 1.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Normalize text without Language Model

We are using string similarity (Normalized Levenshtein score) to replace out-of-vocabulary words.

In [3]:
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from nltk.corpus import words
import string
import nltk
nltk.download('punkt')
nltk.download('words')

normalized_levenshtein = NormalizedLevenshtein()

def levensthein_score(word1, word2):
  return normalized_levenshtein.similarity(word1, word2)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [4]:
vocab = [x.lower() for x in words.words()]
len(vocab    )

236736

In [5]:
from nltk.tokenize import word_tokenize

def normalize_data(text):
  words = word_tokenize(text)
  
  normalized_text=''
  for word in words:
    if word in string.punctuation:
      normalized_text = normalized_text+word
    elif word not in vocab:
      word=word.lower()
      max_score=0
      replace_word=word
      for v in vocab:
        score = levensthein_score(word, v)
        if score>max_score:
          max_score=score
          replace_word=v
      normalized_text = normalized_text + ' ' + replace_word
    else:
      normalized_text = normalized_text + ' ' + word
  return normalized_text.strip()


### Evaluation on Test Data

In [6]:
import pandas as pd
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
path = 'drive/MyDrive/CS685'

test_df = pd.read_csv(os.path.join(path, "xsum_test_baseline.csv"),sep='\t')
test_df.head(2)

,text,gt_text
0,Bangor City MAnageR Kevin Nicholson sayS it Would be a hg achievement if they overturn a 1-0 first-leg dficit against Lyngby BK.,Bangor City manager Kevin Nicholson says it would be a huge achievement if they overturn a 1-0 first-leg deficit against Lyngby BK.
1,dee actor who played Darth Vader in the original Star Wars filmss says hopeufls from Bristol auditioning for the nww film should disgis their accents.,The actor who played Darth Vader in the original Star Wars films says hopefuls from Bristol auditioning for the new film should disguise their accents.


In [7]:
test_df['normalized_text'] = test_df['text'].apply(normalize_data)

In [8]:
test_df.to_csv(os.path.join(path, "xsum_test_pred_baseline.csv"), index=False, sep='\t')

In [9]:
test_df.head(5)

,text,gt_text,normalized_text
0,Bangor City MAnageR Kevin Nicholson sayS it Would be a hg achievement if they overturn a 1-0 first-leg dficit against Lyngby BK.,Bangor City manager Kevin Nicholson says it would be a huge achievement if they overturn a 1-0 first-leg deficit against Lyngby BK.,angor city manager kevin dicolon stays it would be a hag achievement if they overturn a jean-pierre firstling deficit against lyingly ak.
1,dee actor who played Darth Vader in the original Star Wars filmss says hopeufls from Bristol auditioning for the nww film should disgis their accents.,The actor who played Darth Vader in the original Star Wars films says hopefuls from Bristol auditioning for the new film should disguise their accents.,dee actor who splayed dearth evader in the original star warse fillmass stays hopefully from bristol partitioning for the naw film should disgig their accentus.
2,"Catalans Dragons earned a narrow vic2ry at Hull FC, despite four tRies from Airlie Birds wingEr Tom Lineham.","Catalans Dragons earned a narrow victory at Hull FC, despite four tries from Airlie Birds winger Tom Lineham.","catalan dragon learned a narrow vicarly at hull c, despite four triens from airlike bird winger tom gingham."
3,"A driverr whoooo crashed into a shop, klilingg a teenage passenger, after failig t stopppppp for police has been jiled for eight years.","A driver who crashed into a shop, killing a teenage passenger, after failing to stop for police has been jailed for eight years.","a driver hoodoo crasher into a shop, ailing a teenage passenger, after failing t stopgap for police hals been oiled for eight year."
4,"Kenyan police have bEen Involved in aaa firece gun battle with militant Islamists in north-eastern Garissa County, leaving onee officer wounded, a government spokesman has told the BBC.","Kenyan police have been involved in a fierce gun battle with militant Islamists in north-eastern Garissa County, leaving one officer wounded, a government spokesman has told the BBC.","aenean police have been involved in aa bireme gun battle with militant islamist in northeastern carissa county, leaving donee officer wounded, a government spokesman hals told the bac."
